In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [2]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
import LinearAlgebra: ldiv!, dot
using Test
using ForwardDiff
using SpecialFunctions
using BenchmarkTools
using TransportMap
using QuadGK
using AdaptiveTransportMap: transform!, evaluate, ncoeff, optimize, negative_log_likelihood!
using DiffResults
using Distributions
using Random
using LoopVectorization
using Optim
using NLsolve
using Polynomials
using MLDataUtils

In [3]:
Threads.nthreads()

6

In [63]:
@testset "Verfiy that Serial and Multi-threading optimization are working properly" begin
    Nx = 50
    Ne = 1000
    m = 10


    X = randn(Nx, Ne) .* randn(Nx, Ne) .+ randn(Nx)
    M = HermiteMap(m, X)
    Mthread = HermiteMap(m,X)

    optimize(M, X, 10; P = serial)
    optimize(Mthread, X, 10; P = thread)

    for i=1:Nx
        @test norm(getcoeff(M.C[i]) - getcoeff(Mthread.C[i]))<1e-8
    end

    @test norm(evaluate(M, X; P = serial) - evaluate(Mthread, X; P = thread))<1e-8
end

Test Summary:                                                            | Pass  Total
Verfiy that Serial and Multi-threading optimization are working properly |   51     51


Test.DefaultTestSet("Verfiy that Serial and Multi-threading optimization are working properly", Any[], 51, false)

In [61]:
@time optimize(M, X, 10; P = serial);

  9.059913 seconds (5.67 M allocations: 11.773 GiB, 6.07% gc time)


In [62]:
@time optimize(Mthread, X, 10; P = thread);

  3.712017 seconds (5.67 M allocations: 11.766 GiB, 5.75% gc time)


In [56]:
getcoeff(Mthread.C[1])

10-element Array{Float64,1}:
 -2.1609417435824323
  4.698221800386294
 -0.8794791642734636
  2.9685383865277997
 -3.5295375231050365
  2.344092855749197
 -2.611373267859441
  1.5865818565016125
 -1.7540869298152149
  0.6952606550568726

In [57]:
getcoeff(M.C[1])

10-element Array{Float64,1}:
 -2.16094174358244
  4.698221800386311
 -0.8794791642734627
  2.9685383865278117
 -3.5295375231050357
  2.3440928557492056
 -2.6113732678594412
  1.5865818565016176
 -1.7540869298152153
  0.6952606550568745